In [21]:
#1.requirements.txt
!pip install transformers torch pandas scikit-learn numpy gradio

In [22]:
from google.colab import drive
drive.mount('/content/drive')
save_path = '/content/drive/My Drive/Colab Notebooks/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# Define your file paths
save_path = '/content/drive/My Drive/Colab Notebooks/'
train_xlsx = save_path + 'myZoom-train.csv'
eval_xlsx = save_path + 'myZoom-evaluation.csv'
model_save_path = save_path + 'bert_feedback_validator.pt'

In [24]:
# 2. utils/preprocessing.py
import re
import pandas as pd
from transformers import BertTokenizer

def clean_text(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower().strip()
    return text

def preprocess_data(df):
    df['text'] = df['text'].apply(clean_text)
    df['reason'] = df['reason'].apply(clean_text)
    return df

def encode_data(texts, reasons, tokenizer, max_len=128):
    return tokenizer(
        texts,
        reasons,
        padding=True,
        truncation=True,
        max_length=max_len,
        return_tensors='pt'
    )


In [34]:
# 3. train_model.py
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
from torch.optim import AdamW

# --- Load and Preprocess Training Data ---
df_train = pd.read_csv(train_xlsx)
df_pos = df_train.copy()
df_pos['label'] = 1

# Create mismatched text-reason pairs for label 0
df_neg = df_pos.copy()
df_neg['reason'] = df_neg['reason'].sample(frac=1.0, random_state=42).reset_index(drop=True)
df_neg['label'] = 0

# Drop rows where reason accidentally matches original
df_neg = df_neg[df_neg['text'] + df_neg['reason'] != df_pos['text'] + df_pos['reason']]

# Combine both
df_augmented = pd.concat([df_pos, df_neg]).sample(frac=1.0, random_state=42).reset_index(drop=True)

# Save or use this for training
df_train = df_augmented
print(df_train['label'].value_counts())

label
1    2061
0    2061
Name: count, dtype: int64


In [36]:

df_train = preprocess_data(df_train)

# Tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = encode_data(df_train['text'].tolist(), df_train['reason'].tolist(), tokenizer)
train_labels = df_train['label'].tolist()

class FeedbackDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = FeedbackDataset(train_encodings, train_labels)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.train()
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(1):
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1} completed.")


# --- Save Trained Model ---
torch.save(model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 completed.
Model saved to /content/drive/My Drive/Colab Notebooks/bert_feedback_validator.pt


In [37]:
# --- Load Evaluation Data & Evaluate ---
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

df_eval = pd.read_csv(eval_xlsx).sample(n=4122, random_state=42)
df_eval = preprocess_data(df_eval)
eval_encodings = encode_data(df_eval['text'].tolist(), df_eval['reason'].tolist(), tokenizer,max_len=64)
eval_dataset = FeedbackDataset(eval_encodings, df_eval['label'].tolist())
eval_loader = DataLoader(eval_dataset, batch_size=64)


model.eval()
true_labels, pred_labels = [], []

with torch.no_grad():
    for batch in eval_loader:
        inputs = {k: v.to(device) for k, v in batch.items() if k != 'labels'}
        labels = batch['labels'].to(device)
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)
        true_labels.extend(labels.cpu().tolist())
        pred_labels.extend(predictions.cpu().tolist())

# --- Print Metrics ---
print("Evaluation Results:")
print(f"Accuracy: {accuracy_score(true_labels, pred_labels):.4f}")
print(f"Precision: {precision_score(true_labels, pred_labels):.4f}")
print(f"Recall: {recall_score(true_labels, pred_labels):.4f}")
print(f"F1 Score: {f1_score(true_labels, pred_labels):.4f}")

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Evaluation Results:
Accuracy: 0.5080
Precision: 0.4030
Recall: 0.9782
F1 Score: 0.5709


In [38]:
# 4. app/inference.py
# Inference
import gradio as gr

# Load model again if needed
model.load_state_dict(torch.load(model_save_path, map_location='cpu'))
model.eval()

def predict(text, reason):
    inputs = tokenizer(text, reason, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "Aligned (1)" if prediction == 1 else "Not Aligned (0)"

gr.Interface(fn=predict,
             inputs=["text", "text"],
             outputs="text",
             title="Feedback Validator",
             description="Check if user feedback aligns with the dropdown reason.").launch()



It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://45031fe779037723f0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
